In [ ]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Data from https://data.sdss.org/sas/dr17/manga/spectro/mastar/v3_1_1/v1_7_7/
# Data model description https://data.sdss.org/datamodel/files/MANGA_SPECTRO_MASTAR/DRPVER/MPROCVER/mastarall-DRPVER-MPROCVER.html
hdulist = fits.open('data/mastarall-v3_1_1-v1_7_7.fits')
hdulist.info()

dat = Table.read('data/mastarall-v3_1_1-v1_7_7.fits', format='fits', hdu=1)
print('Total stars: %d' % len(dat))

names = [name for name in dat.colnames if len(dat[name].shape) <= 1]
goodstars = dat[names].to_pandas()
goodstars.plot.scatter(x='RA', y='DEC', marker="+", linewidths=0.5, figsize=(15,7))
plt.show()

print("Vaues in PHOTOCAT:")
print(goodstars['PHOTOCAT'].value_counts())

print("Data points with valid T value from catalogue:")
goodstars['INPUT_TEFF'].describe()
tstars = goodstars[goodstars['INPUT_TEFF']>0.0]
print(tstars['INPUT_TEFF'].describe())

print("Histogram of good-T stars:")
tstars['INPUT_TEFF'].hist()
plt.show()

print("Random good-T star:")
goodstars.sample(n=5)

One can look up the actual image via e.g. http://skyserver.sdss.org/dr17/VisualTools/explore/summary?ra=48.873343&dec=-8.442834 (need RA and DEC).

Caution is required interpreting the spectra, as resolution differs per star and even per visit https://www.sdss.org/dr17/mastar/mastar-spectra/. SDSS team provides "unified resolution" spectra (with resolutions adjusted to match). Best one to use then should be the "combined" dataset (multiple visits combined to increase quality) and the unified 99.5th set (which is the largest). I think we don't really care about the shape of these LSF curves, as long as they are comparable.

Alternatively, we can use raw data and feed our algorithm with resolution data too. This could give better results because I think normalisation removes detail.

In [ ]:
# mastarall-gaiaedr3-extcorr-simbad-ps1-v3_1_1-v1_7_7-v1.fits contains some additional columns taken from gaia catalogue, such as parallax
# Data def is here https://data.sdss.org/datamodel/files/MANGA_SPECTRO_MASTAR/DRPVER/MPROCVER/vac/crossmatch/VER/mastarall-gaiaedr3-extcorr-simbad-ps1-DRPVER-MPROCVER-VER.html
hdulist = fits.open('data/mastarall-gaiaedr3-extcorr-simbad-ps1-v3_1_1-v1_7_7-v1.fits')
hdulist.info()
dat = Table.read('data/mastarall-gaiaedr3-extcorr-simbad-ps1-v3_1_1-v1_7_7-v1.fits', format='fits', hdu=1)
dat[[0,1,2]]

In [ ]:
# mastar-goodstars-v3_1_1-v1_7_7-params-v1.fits contains params derived by different researchers
# Data def is here https://data.sdss.org/datamodel/files/MANGA_SPECTRO_MASTAR/DRPVER/MPROCVER/vac/parameters/VER/mastar-goodstars-DRPVER-MPROCVER-params-VER.html
hdulist = fits.open('data/mastar-goodstars-v3_1_1-v1_7_7-params-v1.fits')
hdulist.info()
dat = Table.read('data/mastar-goodstars-v3_1_1-v1_7_7-params-v1.fits', format='fits', hdu=1)
dat[[0,1,2]]

In [ ]:
# As mentioned before, I think using normalised spectra makes most sense, unless we expect our method to be able to deal with resolution variability.
# Therefore first try using normalised mastar-combspec-v3_1_1-v1_7_7-lsfpercent99.5.fits
# Data def is here https://data.sdss.org/datamodel/files/MANGA_SPECTRO_MASTAR/DRPVER/MPROCVER/mastar-combspec-DRPVER-MPROCVER-lsfpercentPP.P.html
hdulist = fits.open('data/mastar-combspec-v3_1_1-v1_7_7-lsfpercent99.5.fits')
hdulist.info()
spectra = Table.read('data/mastar-combspec-v3_1_1-v1_7_7-lsfpercent99.5.fits', format='fits', hdu=1)
spectra[[0,1,2]]

Spectral data consists of: WAVE (frequency value - this is the same for all entries, an x-axis value), FLUX (amount of light reaching the detector, IVAR is the inverse squared, this is the y value), DISP (resolution-related, also PREDISP), MASK (for data goodness?).

When using LFS normalised data, we can ignore DISP (they have all been normalised to the same value).

IVAR is somewhat [described here](https://www.sdss.org/dr14/manga/manga-tutorials/manga-faq/#WhydoyououtputIVAR(inversevariance)insteadoferrors?). Infinite flux is represented by 0.

We can join these spectrums with our main data table via MANGAID.

In [ ]:
for s in np.random.choice(spectra, size=10):
	plt.plot(s['FLUX'])

This looks good, although:

* some datapoints are negative
* there is some significant dips in the flux values - are these the absorption lines?

Some more info from [MaNGA IDL tutorial](https://www.sdss.org/dr13/manga/manga-tutorials/how-do-i-look-at-my-data/idl/):

* When mask does not equal 0, this indicates a bad pixel. Ivar stands for the inverse variance.
* whenever ivar is 0, this also indicates a bad pixel.

Hopefully these values will help us remove the bad values. So let's put the algorithm togheter:

* zero the values where IVAR is zero or MASK is non-zero
* feed the resulting spectral data (FLUX) into the algorithm
* target the TEFF_INPUT label via join on MANGAID
* initially use a data split on goodstars containing a valid TEFF taken from upstream catalogues (caution is needed, because multiple catalogues are used). There is about 4,000 of these stars.
* we can then further test the algorithm on the extra data provided in the "params" set by comparing how it fared compared to other algorithms (24,290 stars in total)

Alternatively, if 4,000 stars is not enough for training, we can pick one of the values produced by other algorithms (the less-ML) ones, and we can still judge results against other algorithms.
